In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 패키지 설치
!pip install mxnet==1.6.0 gluonnlp==0.9.1 sentencepiece==0.1.91 pandas==1.0.5 transformers==2.11.0 pytorch_lightning==0.8.1 onnx onnxruntime coremltools

     |████████████████████████████████| 3.4MB 4.3MB/s 
  Created wheel for attr: filename=attr-0.3.1-cp36-none-any.whl size=2459 sha256=5b42000be0dd70325adeeb4e1a4b5acf1c665ee199c8ae2f4d7ce718a699a23c
  Stored in directory: /root/.cache/pip/wheels/f0/96/9b/1f8892a707d17095b5a6eab0275da9d39e68e03a26aee2e726
Successfully built attr


In [8]:
# 폴더 이동
%cd /content/drive/MyDrive/Colab Notebooks/gpt/KoGPT2-chatbot-master

import torch
import numpy as np
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from pytorch_lightning.core.lightning import LightningModule
import coremltools as ct

tok_path = get_tokenizer()
model, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path, num_best=0, alpha=0)

class KoGPT2Chat(LightningModule):
    def __init__(self, hparams, **kwargs):
        super(KoGPT2Chat, self).__init__()
        self.hparams = hparams
        self.tok_path = tok_path
        self.neg = -1e18
        self.kogpt2, self.vocab = model, vocab

        self.loss_function = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs):
        # (batch, seq_len, hiddens)
        output, _ = self.kogpt2(inputs)
        return output

# init
U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '<s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'

gpt_chat_model = KoGPT2Chat.load_from_checkpoint("./model_chp/model_last.ckpt")
gpt_chat_model.eval()

sent_tokens = tok("")
sent='0'
toked = tok(sent)
q_tok = tok("")
a = ''
a_tok = []

input_ids = torch.LongTensor([
                    vocab[U_TKN]] + vocab[q_tok] +
                    vocab[EOS, SENT] + vocab[sent_tokens] +
                    vocab[EOS, S_TKN] +
                    vocab[a_tok]).unsqueeze(dim=0)

# export
traced_model = torch.jit.trace(gpt_chat_model, input_ids)
traced_model.save("sample.pt")

input_shape = ct.Shape(shape=(1, ct.RangeDim(), 512))
model_input = ct.TensorType(shape=input_shape, dtype=np.int16)

# Convert the saved PyTorch model to Core ML
mlmodel = ct.convert(traced_model, inputs=[model_input])
mlmodel.save('./mlmodel.mlmodel')
print("Export of kogpt2 mlmodel complete!")

/content/drive/MyDrive/Colab Notebooks/gpt/KoGPT2-chatbot-master
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:146: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  w = w / (float(v.size(-1)) ** 0.5)
/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:148: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  mask = self.bias[:, :, ns - nd : ns, :ns]
Translating MIL ==> MLModel Ops: 100%|██████████| 1316/1316 [00:26<00:00, 50.06 ops/s]


Export of kogpt2 mlmodel complete!
